In [17]:
from binance.client import Client
import pandas as pd
import plotly.graph_objects as go
from pon import api,secret
import numpy as np
import pandas_ta as ta
import ccxt
import warnings
warnings.filterwarnings("ignore")

In [18]:
def get_price_data_binance(ticker:str,limit:int)->pd.DataFrame:
    client = Client(api, secret)
    interval = Client.KLINE_INTERVAL_1DAY
    df = pd.DataFrame(client.get_historical_klines(symbol=ticker, interval=interval, limit=limit))
    df.columns=['date','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df = df.drop(['close_time','d1','d2','d3','d4','d5'],axis=1)
    df['date'] = pd.to_datetime(df['date']*1000000)
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    return df



In [26]:
def get_list_of_up_imbalances(df:pd.DataFrame)->list:
    up_imbalances = []
    for i in range(2, len(df)):
        prev_candle = df.iloc[i-2]
        next_candle = df.iloc[i]
        if prev_candle['high'] < next_candle['low']:
            up_imbalances.append([round((prev_candle['high']+next_candle['low'])/2,2), next_candle['low'],df.iloc[i-1]['date']])
    return up_imbalances

def get_list_of_down_imbalances(df:pd.DataFrame)->list:
    down_imbalances = []
    for i in range(2, len(df)):
        prev_candle = df.iloc[i-2]
        next_candle = df.iloc[i]
        if prev_candle['low'] > next_candle['high']:
            down_imbalances.append([round((prev_candle['low']+next_candle['high'])/2,2) , prev_candle['low'] , df.iloc[i-1]['date']])
    return down_imbalances

def delete_up_imbalance(up_imbalances, df:pd.DataFrame)->list:
    for i in range(0, len(df)):
        for imbalance in up_imbalances:
            candle = df.iloc[i]
            if candle['date']>imbalance[2] and candle['open'] >= imbalance[0] and (candle['close'] < imbalance[0] or candle['low'] < imbalance[0]):
                up_imbalances.remove(imbalance)
    return up_imbalances

def delete_down_imbalance(down_imbalances, df:pd.DataFrame)->list:
    for i in range(0, len(df)):
        for imbalance in down_imbalances:
            candle = df.iloc[i]
            if candle['date']>imbalance[2] and candle['open'] <= imbalance[0] and (candle['close'] > imbalance[0] or candle['high'] > imbalance[0]):
                down_imbalances.remove(imbalance)
    
    return down_imbalances

def add_moving_average(period:int,df:pd.DataFrame)->pd.DataFrame:
    df['SMA'] = df['close'].rolling(period).mean()
    df['SMAslope'] = df['SMA'].pct_change()*100

    df['up_reversal'] = ((df['SMAslope'].shift(1) < 0) & (df['SMAslope'] > 0)) & ((df['SMAslope'].shift(1) < -0.7) | (df['SMAslope'].shift(2) < -0.7) | (df['SMAslope'].shift(3) < -0.7) | (df['SMAslope'].shift(4) < -0.7) | (df['SMAslope'].shift(5) < -0.7)) & ((df['SMAslope'].shift(-1) > 0.7) | (df['SMAslope'].shift(-2) > 0.7) | (df['SMAslope'].shift(-3) > 0.7) | (df['SMAslope'].shift(-4) > 0.7) | (df['SMAslope'].shift(-5) > 0.7))
    df['down_reversal'] = ((df['SMAslope'].shift(1) > 0) & (df['SMAslope'] < 0)) & ((df['SMAslope'].shift(2) > 0.7) | (df['SMAslope'].shift(2) > 0.7) | (df['SMAslope'].shift(3) > 0.7) | (df['SMAslope'].shift(4) > 0.7) | (df['SMAslope'].shift(5) > 0.7)) & ((df['SMAslope'].shift(-1) < -0.7) | (df['SMAslope'].shift(-2) < -0.7) | (df['SMAslope'].shift(-3) < -0.7) | (df['SMAslope'].shift(-4) < -0.7) | (df['SMAslope'].shift(-5) < -0.7))

    df['charthighest'] = df['high'].rolling(6).max()[df['down_reversal'] == True]
    df['chartlowest'] = df['low'].rolling(6).min()[df['up_reversal'] == True]

    return df

def add_rsi(period:int,df:pd.DataFrame)->pd.DataFrame:
    df["RSI"] = ta.rsi(round((df['close']+df['high']+df['low'])/3,2), period)
    df['RSIslope'] = df['RSI'].pct_change()

    return df


In [33]:
df = get_price_data_binance('BTCUSDT',1000)



In [34]:
dfslice = df[800:]

In [35]:
up_imbalances = get_list_of_up_imbalances(dfslice)
down_imbalances = get_list_of_down_imbalances(dfslice)
# up_imbalances = delete_up_imbalance(up_imbalances,dfslice)
# up_imbalances = delete_down_imbalance(down_imbalances,dfslice)
df = add_moving_average(5,dfslice)
df = add_rsi(21,dfslice)
# up_imbalances.sort(key=lambda x: x[2])
# down_imbalances.sort(key=lambda x: x[2])

In [36]:
fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.add_trace(
    go.Scatter(
        x=df['date'],
        y=df['SMA']
    ))

fig1 = go.Figure(go.Scatter(
        x=df['date'],
        y=df['SMAslope']
    ))

# rsi = go.Figure(go.Scatter(
#         x=df['date'],
#         y=df['RSI']
#     ))
for i in range(0, len(df)):
    if df.iloc[i]['up_reversal']:
        fig.add_vline(df.iloc[i]['date'],line_color="green")
    if df.iloc[i]['down_reversal']:
        fig.add_vline(df.iloc[i]['date'],line_color="red")

fig.add_trace(go.Scatter(x=df['date'], y=df['charthighest'],connectgaps=True))
fig.add_trace(go.Scatter(x=df['date'], y=df['chartlowest'],connectgaps=True))


    # candle = df.iloc[i]
    # if candle['down_reversal']:
    #     window = df.rolling(window=3)

    #     # Use the max function to get the maximum value in the window
    #     aboba = window['high'].max()
    #     for imbalance in down_imbalances:
    #         if candle['date']>imbalance[2] and imbalance[0]<aboba[i]<imbalance[1]:
    #             print(imbalance[0],aboba[i],imbalance[1],candle['date'],imbalance[2])
    #             fig.add_vline(candle['date'])

# for imbalance in up_imbalances:
#     fig.add_trace(go.Scatter(x=[imbalance[2]], y=[imbalance[0]]))
# for imbalance in down_imbalances:
#     fig.add_trace(go.Scatter(x=[imbalance[2]], y=[imbalance[0]]))

fig.update_layout(showlegend=False)
fig.show()
fig1.update_layout(showlegend=False)
fig1.show()

# rsi.update_layout(showlegend=False)
# rsi.show()
